# Score

> Score functions and classes

In [ ]:
#| default_exp score

In [ ]:
#| hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
from emb_opt.imports import *
from emb_opt.module import Module
from emb_opt.schemas import Item, Query, Batch, ScoreFunction, ScoreResponse

/home/dmai/miniconda3/envs/emb_opt/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The Score step assigns a numeric score to each `Item` result. This score is used to drive the hill climbing algorithm. The score step is formalized by the `ScoreFunction` schema, which maps inputs `List[Item]` to outputs `List[ScoreResponse]`.

The `ScoreModule` manages execution of a `ScoreFunction`. The `ScoreModule` gathers valid items, sends them to the `ScoreFunction`, and processes the results.

In [ ]:
#| export

class ScoreModule(Module):
    def __init__(self, 
                 function: ScoreFunction # score function
                ):
        super().__init__(ScoreResponse, function)
        
    def gather_inputs(self, batch: Batch) -> (List[Tuple], List[Item]):
        idxs, inputs = batch.flatten_query_results()
        return (idxs, inputs)
    
    def scatter_results(self, batch: Batch, idxs: List[Tuple], results: List[ScoreResponse]):
        for (q_idx, r_idx), result in zip(idxs, results):
            batch_item = batch.get_item(q_idx, r_idx)
            
            batch_item.score = result.score
            
            if result.data:
                batch_item.data.update(result.data)
                
            if not result.valid:
                batch_item.update_internal(removed=True, removal_reason='score response invalid')
                
        for _, result in batch.enumerate_query_results():
            if (result.score is None) and (not result.internal.removed):
                result.update_internal(removed=True, removal_reason='null score')
                
        for query in batch:
            query.update_internal()

In [ ]:
def build_batch():
    d_emb = 128
    n_emb = 100
    np.random.seed(42)
    
    embeddings = np.random.randn(n_emb+1, d_emb)
    query = Query.from_minimal(embedding=embeddings[-1])
    results = [Item.from_minimal(id=i, embedding=embeddings[i]) for i in range(n_emb)]
    query.add_query_results(results)
    batch = Batch(queries=[query])
    
    expected_scores = [np.linalg.norm(i.embedding) for i in results]
    return batch, expected_scores

class NormScore():
    def __init__(self, test_fails, test_nulls):
        self.test_fails = test_fails
        self.test_nulls = test_nulls
        
    def __call__(self, inputs: List[Item]) -> List[ScoreResponse]:
        embeddings = np.array([i.embedding for i in inputs])
        norms = np.linalg.norm(embeddings, axis=-1)
        
        if self.test_fails:
            results = [ScoreResponse(valid=False, score=i, data={'norm':i}) for i in norms]
        elif self.test_nulls:
            results = [ScoreResponse(valid=True, score=None if np.random.uniform()<0.5 else i, 
                                     data={'norm':i}) for i in norms]
        else:
            results = [ScoreResponse(valid=True, score=i, data={'norm':i}) for i in norms]
            
        return results
    
batch, scores = build_batch()
score_func = NormScore(False, False)
score_module = ScoreModule(score_func)
batch = score_module(batch)
assert np.allclose([i.score for i in batch[0]], scores)

batch, scores = build_batch()
score_func = NormScore(True, False)
score_module = ScoreModule(score_func)
batch = score_module(batch)
assert batch[0].internal.removed

batch, scores = build_batch()
score_func = NormScore(False, True)
score_module = ScoreModule(score_func)
batch = score_module(batch)

for _, result in batch.enumerate_query_results(skip_removed=False):
    if result.score is None:
        assert result.internal.removed
    else:
        assert not result.internal.removed

In [ ]:
#| export

class ScorePlugin():
    '''
    ScorePlugin - documentation for plugin functions to `ScoreFunction`
    
    A valid `ScoreFunction` is any function that maps `List[Item]` to
    `List[ScoreResponse]`. The inputs will be given as `Item` objects. 
    The outputs can be either a list of `ScoreResponse` objects or a list 
    of valid json dictionaries that match the `ScoreResponse` schema.
    
    Item schema:
    
    `{
        'id' : Optional[Union[str, int]]
        'item' : Optional[Any],
        'embedding' : List[float],
        'score' : None, # will be None at this stage
        'data' : Optional[Dict],
    }`
    
    Input schema:
    
    `List[Item]`
    
    ScoreResponse schema:

    `{
        'valid' : bool,
        'score' : Optional[float], # can be None if valid=False
        'data' : Optional[Dict],
    }`
    
    Output schema:
    
    `List[ScoreResponse]`
    
    '''
    def __call__(self, inputs: List[Query]) -> List[ScoreResponse]:
        pass